In [1]:
using CUDA
using TypedTables
using DataFrames
using Statistics
using CSV

In [2]:
# CUDA

function operation_cuda(A, B)
    C = CUDA.zeros(Float16, size(A))
    CUDA.@sync C .= A .- B
    return C
end

function simulate_gpu(A_normal, B_normal)
    # Convert your Float64 matrices to Float32 and then to CuArray{Float32}
    A = CUDA.convert(CuArray{Float16}, A_normal)
    B = CUDA.convert(CuArray{Float16}, B_normal)
    # Mengukur waktu eksekusi
    cuda_start = CUDA.@elapsed result_gpu = operation_cuda(A, B)
    return cuda_start
end

simulate_gpu (generic function with 1 method)

In [3]:
# CPU

function operation_cpu(A, B)
    C = zeros(Float16, size(A))
    C .= A .- B
    return C
end

function simulate_cpu(A_normal, B_normal)
    normal_start = @elapsed result_cpu = operation_cpu(A_normal, B_normal)
    return normal_start
end

simulate_cpu (generic function with 1 method)

In [4]:
function simulate(N::Int64, n_loop::Int64)
    println("Total size: $(N*N)")
    results_gpu = []
    results_cpu = []
    is_better_gpu = []
    for i in 1:n_loop
        A_normal = rand(Float16, N, N)
        B_normal = rand(Float16, N, N)
        benchmark_gpu = simulate_gpu(A_normal, B_normal)
        benchmark_cpu = simulate_cpu(A_normal, B_normal)
        push!(results_gpu, benchmark_gpu)
        push!(results_cpu, benchmark_cpu)
        push!(is_better_gpu, benchmark_gpu < benchmark_cpu)
    end
    println("Mean CPU: $(mean(results_cpu))")
    println("Mean GPU: $(mean(results_gpu))")
    return DataFrame(CPU=results_cpu, GPU=results_gpu, Better_GPU=is_better_gpu)
end

simulate (generic function with 1 method)

In [8]:
df = simulate(10, 10)
CSV.write("./1.csv", df)
df

Total size: 100
Mean CPU: 6.205e-7
Mean GPU: 1.99808e-5


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,1.744e-6,4.096e-5,false
2,6.25e-7,2.4576e-5,false
3,5.37e-7,1.7408e-5,false
4,4.58e-7,1.7408e-5,false
5,5.02e-7,1.7408e-5,false
6,4.58e-7,1.7408e-5,false
7,4.63e-7,1.6384e-5,false
8,4.68e-7,1.6384e-5,false
9,4.89e-7,1.5488e-5,false


In [9]:
df = simulate(50, 10)
CSV.write("./2.csv", df)
df

Total size: 2500
Mean CPU: 2.9070999999999997e-6
Mean GPU: 2.7868798e-5


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,3.729e-6,4.608e-5,false
2,2.135e-6,2.56e-5,false
3,2.998e-6,2.3552e-5,false
4,2.982e-6,2.7648e-5,false
5,2.589e-6,2.6752e-5,false
6,4.019e-6,2.9696e-5,false
7,2.971e-6,2.6624e-5,false
8,2.747e-6,2.56e-5,false
9,2.948e-6,2.56e-5,false


In [10]:
df = simulate(100, 10)
CSV.write("./3.csv", df)
df

Total size: 10000
Mean CPU: 6.455800000000002e-6
Mean GPU: 2.0992e-5


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,5.679e-6,4.608e-5,false
2,4.379e-6,1.9456e-5,false
3,4.279e-6,1.7408e-5,false
4,4.757e-6,1.8432e-5,false
5,2.1398e-5,1.9456e-5,true
6,4.618e-6,1.7408e-5,false
7,4.695e-6,1.9456e-5,false
8,4.283e-6,1.8432e-5,false
9,4.268e-6,1.7408e-5,false


In [14]:
df = simulate(500, 10)
CSV.write("./4.csv", df)
df

Total size: 250000
Mean CPU: 0.00024136970000000006
Mean GPU: 9.648e-5


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,0.000141311,9.4592e-5,true
2,0.000359101,9.7632e-5,true
3,0.000319786,9.2992e-5,true
4,0.000743177,0.000105408,true
5,0.000273667,0.000100736,true
6,0.000115695,9.7632e-5,true
7,0.000125933,9.3248e-5,true
8,0.000116413,9.6928e-5,true
9,0.000103765,9.0816e-5,true


In [16]:
df = simulate(1000, 10)
CSV.write("./5.csv", df)
df

Total size: 1000000
Mean CPU: 0.000716617
Mean GPU: 0.00038913923


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,0.000872844,0.000405984,true
2,0.000632758,0.000387488,true
3,0.000542304,0.00037984,true
4,0.00040746,0.000393344,true
5,0.00050934,0.000364064,true
6,0.000423035,0.000391936,true
7,0.00224217,0.000414784,true
8,0.000365828,0.000388352,false
9,0.000477458,0.000385216,true


In [17]:
df = simulate(5000, 10)
CSV.write("./6.csv", df)
df

Total size: 25000000
Mean CPU: 0.058224026400000004
Mean GPU: 0.008563207


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,0.0761132,0.00861242,true
2,0.0300125,0.00851315,true
3,0.0296853,0.00859418,true
4,0.0267174,0.0084665,true
5,0.0277874,0.00861402,true
6,0.265931,0.00861664,true
7,0.0328643,0.00858131,true
8,0.033966,0.00850976,true
9,0.0311682,0.0085088,true


In [18]:
df = simulate(10000, 10)
CSV.write("./7.csv", df)
df

Total size: 100000000
Mean CPU: 0.13268751930000003
Mean GPU: 0.034584127


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,0.161321,0.0351873,true
2,0.10758,0.034346,true
3,0.104388,0.0342921,true
4,0.1047,0.0346456,true
5,0.104314,0.0349704,true
6,0.321986,0.0345399,true
7,0.106519,0.0342132,true
8,0.106361,0.0345646,true
9,0.105267,0.0346173,true
